# Relevant imports and Groq Clients 

In [107]:
import os
import re
import math
import json
from dotenv import load_dotenv 

In [108]:
%cd ..

/home


In [109]:
from groq import Groq
from src.agentic_patterns.tool_pattern.tool import tool
from src.agentic_patterns.utils.extractions import extract_tag_content

load_dotenv()

False

In [110]:
MODEL = "llama-3.3-70b-versatile"

In [111]:
GROQ_CLIENT = Groq()

## A system prompt for ReAct Loop

Như đã thực hiện với Tool Pattern, Hệ thống này cũng cần một System Prompt cho kỹ thuật ReAct. System Prompt này sẽ mô tả ReAct loop.

1. Thought: The LLM will think about which action to take.
2. Action: The LLM will use a Tool to "act on the environment".
3. Observation: The LLM will observe the tool output and reflect on the next thing to do.

Một điểm khác biệt chính so với  Tool Pattern System Prompt là chúng tôi sẽ đính kèm tất cả thông báo bằng các thẻ, như sau: Chúng tôi triển khai logic ReAct trên nền tảng đám mây mà không cần các thẻ này, nhưng tôi thấy LLM dễ dàng hiểu hướng dẫn theo cách này hơn. 


In [112]:
REACT_SYSTEM_PROMPT = """
Bạn là một Function calling AI model. Bạn hoạt động theo một vòng lặp gồm 3 bước: Thought, Action, Observation.
Bạn được cung cấp một Function signatures với <tools></tools> XML tags.
Bạn có thể gọi một hoặc nhiều functions để hỗ trợ cùng với user query. Không được đưa ra những giả định về những giá trị nào sẽ được đưa vào Functions. Đặc biệt cần chú ý tới các đặc tính 'types'. Bạn nên sử dụng types như trong một lệnh Python.

Mỗi Function Call trả về một đối tượng JSON cùng với Function Name và các Arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{"name":<function-name>, "arguments":"args-dict", "id":<monotonically-increasing-id>}
</tool_call>

Ở đây có sẵn tools / actions:

<tools>
%s
</tools>

Example session:
<question> Nhiệt độ hiện tại của Hà Nội là bao nhiêu?</question>
<thought>Tôi cần tới get the current weather tại Hà Nội</thought>
<tool_call>{"name": "get_current_weather", "arguments": {"location": "Hà Nội", "unit": "C"}, "id": 0}</tool_call>

Bạn sẽ phải gọi lại điều này:

<observation> {0: {"temperature": 25, "unit": "C"}} </observation>

Bạn trả về output:
<response> Hiện tại nhiệt độ của Hà Nội là 25 độ C </response>

Các ràng buộc bổ sung:

Nếu người dùng yêu cầu về những điều không liên quan tới tools ràng buộc, hãy trả lời tự do theo cách của bạn kèm theo <response></response> tags XML.
"""

In [113]:
@tool
def sum_two_elements(a: int, b: int) -> int:
    """
    Computes the sum of two integers.
    Args:
        a (int): Số nguyên thứ nhất để tính tổng.
        b (int): Số nguyên thứ hai để tính tổng.

    Returns:
        int: Tổng của hai số nguyên a và b
    """
    return a+b

@tool
def multiply_two_elements(a: int, b: int) -> int:
    """
    Computes the sum of two integers.
    Args:
        a (int): Số nguyên thứ nhất để tính tích.
        b (int): Số nguyên thứ hai để tính tích.

    Returns:
        int: Tích của hai số nguyên a và b
    """
    return a*b

@tool
def compute_log(x: int) -> float | str:
    """
    Computes the log of an integers x cùng với một optional base.
    Args:
        x (int): Số nguyê để tính log.

    Returns:
        float: logarithm của x.
    """
    if x <= 0:
        return "Logarithm is underfined for values less than or equal to 0."
    return math.log(x)

In [114]:
available_tools = {
    "sum_two_elements" : sum_two_elements,
    "multiply_two_elements": multiply_two_elements,
    "compute_log": compute_log
}

In [115]:
print("Tool name: ", sum_two_elements.name)
print("Tool signature: ", sum_two_elements.fn_signature)

Tool name:  sum_two_elements
Tool signature:  {"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n    Args:\n        a (int): S\u1ed1 nguy\u00ean th\u1ee9 nh\u1ea5t \u0111\u1ec3 t\u00ednh t\u1ed5ng.\n        b (int): S\u1ed1 nguy\u00ean th\u1ee9 hai \u0111\u1ec3 t\u00ednh t\u1ed5ng.\n\n    Returns:\n        int: T\u1ed5ng c\u1ee7a hai s\u1ed1 nguy\u00ean a v\u00e0 b\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}}


In [116]:
tools_signature = sum_two_elements.fn_signature + ",\n" + multiply_two_elements.fn_signature + ",\n" + compute_log.fn_signature

In [117]:
print(tools_signature)

{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n    Args:\n        a (int): S\u1ed1 nguy\u00ean th\u1ee9 nh\u1ea5t \u0111\u1ec3 t\u00ednh t\u1ed5ng.\n        b (int): S\u1ed1 nguy\u00ean th\u1ee9 hai \u0111\u1ec3 t\u00ednh t\u1ed5ng.\n\n    Returns:\n        int: T\u1ed5ng c\u1ee7a hai s\u1ed1 nguy\u00ean a v\u00e0 b\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},
{"name": "multiply_two_elements", "description": "\n    Computes the sum of two integers.\n    Args:\n        a (int): S\u1ed1 nguy\u00ean th\u1ee9 nh\u1ea5t \u0111\u1ec3 t\u00ednh t\u00edch.\n        b (int): S\u1ed1 nguy\u00ean th\u1ee9 hai \u0111\u1ec3 t\u00ednh t\u00edch.\n\n    Returns:\n        int: T\u00edch c\u1ee7a hai s\u1ed1 nguy\u00ean a v\u00e0 b\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},
{"name": "compute_log", "description": "\n    Computes the log of an integers x c\u00f9ng v\u1edbi m\u1ed9t opt

In [118]:
REACT_SYSTEM_PROMPT = REACT_SYSTEM_PROMPT % tools_signature # Gán system prompt với tools signature

In [119]:
print(REACT_SYSTEM_PROMPT)


Bạn là một Function calling AI model. Bạn hoạt động theo một vòng lặp gồm 3 bước: Thought, Action, Observation.
Bạn được cung cấp một Function signatures với <tools></tools> XML tags.
Bạn có thể gọi một hoặc nhiều functions để hỗ trợ cùng với user query. Không được đưa ra những giả định về những giá trị nào sẽ được đưa vào Functions. Đặc biệt cần chú ý tới các đặc tính 'types'. Bạn nên sử dụng types như trong một lệnh Python.

Mỗi Function Call trả về một đối tượng JSON cùng với Function Name và các Arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{"name":<function-name>, "arguments":"args-dict", "id":<monotonically-increasing-id>}
</tool_call>

Ở đây có sẵn tools / actions:

<tools>
{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n    Args:\n        a (int): S\u1ed1 nguy\u00ean th\u1ee9 nh\u1ea5t \u0111\u1ec3 t\u00ednh t\u1ed5ng.\n        b (int): S\u1ed1 nguy\u00ean th\u1ee9 hai \u0111\u1ec3 t\u00ednh t\u1ed5ng.\n\n    Re

# ReAct Loop Step 1

In [120]:
USER_QUESTION = "Tôi muốn tính toán tổng của hai số 1234 với 5678 và nhân kết quả đó với 5. Sau đó tôi muốn tính logarithms cho kết quả đó"


In [121]:
chat_history = [
    {
        "role": "system",
        "content": REACT_SYSTEM_PROMPT,
    },
    {
        "role": "user",
        "content": USER_QUESTION,
    }
]

In [122]:
output = GROQ_CLIENT.chat.completions.create(
    messages = chat_history,
    model = MODEL
).choices[0].message.content

In [123]:
print(output)

Để thực hiện yêu cầu của bạn, tôi sẽ thực hiện các bước sau:

1. Tính tổng của hai số 1234 và 5678.
2. Nhân kết quả đó với 5.
3. Tính logarithm của kết quả.

Tôi sẽ bắt đầu bằng cách gọi hàm `sum_two_elements` để tính tổng của hai số 1234 và 5678.

<tool_call>{"name": "sum_two_elements", "arguments": {"a": 1234, "b": 5678}, "id": 0}</tool_call>

Đợi trả về kết quả... 

Sau khi nhận được kết quả, tôi sẽ tiếp tục gọi hàm `multiply_two_elements` để nhân kết quả đó với 5.

<tool_call>{"name": "multiply_two_elements", "arguments": {"a": {"function_name": "sum_two_elements", "args": [1234, 5678]}, "b": 5}, "id": 1}</tool_call>

Đợi trả về kết quả... 

Cuối cùng, tôi sẽ gọi hàm `compute_log` để tính logarithm của kết quả.

<tool_call>{"name": "compute_log", "arguments": {"x": {"function_name": "multiply_two_elements", "args": [{"function_name": "sum_two_elements", "args": [1234, 5678]}, 5]}}, "id": 2}</tool_call>

Đợi trả về kết quả... 

Sau khi nhận được tất cả kết quả, tôi sẽ trả về output.

In [124]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

## ReAct Loop Step 2

In [125]:
tool_call = extract_tag_content(output, tag="tool_call")

In [126]:
tool_call

TagContentResult(content=['{"name": "sum_two_elements", "arguments": {"a": 1234, "b": 5678}, "id": 0}', '{"name": "multiply_two_elements", "arguments": {"a": {"function_name": "sum_two_elements", "args": [1234, 5678]}, "b": 5}, "id": 1}', '{"name": "compute_log", "arguments": {"x": {"function_name": "multiply_two_elements", "args": [{"function_name": "sum_two_elements", "args": [1234, 5678]}, 5]}}, "id": 2}'], found=True)

In [127]:
tool_call = json.loads(tool_call.content[0])

In [128]:
tool_call

{'name': 'sum_two_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 0}

In [129]:
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [130]:
assert tool_result == 1234 + 5678

In [131]:
print(tool_result)

6912


In [132]:
chat_history.append(
    {
        "role": "user",
        "content":f"<observation>{tool_result}</observation>"
    }
)

## ReAct Loop Step 3

In [133]:
output = GROQ_CLIENT.chat.completions.create(
    messages = chat_history,
    model = MODEL
).choices[0].message.content

print(output)


Tôi đã nhận được kết quả từ hàm `sum_two_elements`. Kết quả là 6912.

Tiếp theo, tôi sẽ gọi hàm `multiply_two_elements` để nhân kết quả đó với 5.

<tool_call>{"name": "multiply_two_elements", "arguments": {"a": 6912, "b": 5}, "id": 1}</tool_call>


In [134]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

## ReAct Loop Step 4

In [135]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])

tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [136]:
tool_result

34560

In [137]:
chat_history

[{'role': 'system',
  'content': '\nBạn là một Function calling AI model. Bạn hoạt động theo một vòng lặp gồm 3 bước: Thought, Action, Observation.\nBạn được cung cấp một Function signatures với <tools></tools> XML tags.\nBạn có thể gọi một hoặc nhiều functions để hỗ trợ cùng với user query. Không được đưa ra những giả định về những giá trị nào sẽ được đưa vào Functions. Đặc biệt cần chú ý tới các đặc tính \'types\'. Bạn nên sử dụng types như trong một lệnh Python.\n\nMỗi Function Call trả về một đối tượng JSON cùng với Function Name và các Arguments within <tool_call></tool_call> XML tags as follows:\n<tool_call>\n{"name":<function-name>, "arguments":"args-dict", "id":<monotonically-increasing-id>}\n</tool_call>\n\nỞ đây có sẵn tools / actions:\n\n<tools>\n{"name": "sum_two_elements", "description": "\\n    Computes the sum of two integers.\\n    Args:\\n        a (int): S\\u1ed1 nguy\\u00ean th\\u1ee9 nh\\u1ea5t \\u0111\\u1ec3 t\\u00ednh t\\u1ed5ng.\\n        b (int): S\\u1ed1 nguy\\

In [138]:
chat_history.append(
    {
        "role": "user",
        "content":f"<observation>{tool_result}</observation>"
    }
)

In [141]:
chat_history

[{'role': 'system',
  'content': '\nBạn là một Function calling AI model. Bạn hoạt động theo một vòng lặp gồm 3 bước: Thought, Action, Observation.\nBạn được cung cấp một Function signatures với <tools></tools> XML tags.\nBạn có thể gọi một hoặc nhiều functions để hỗ trợ cùng với user query. Không được đưa ra những giả định về những giá trị nào sẽ được đưa vào Functions. Đặc biệt cần chú ý tới các đặc tính \'types\'. Bạn nên sử dụng types như trong một lệnh Python.\n\nMỗi Function Call trả về một đối tượng JSON cùng với Function Name và các Arguments within <tool_call></tool_call> XML tags as follows:\n<tool_call>\n{"name":<function-name>, "arguments":"args-dict", "id":<monotonically-increasing-id>}\n</tool_call>\n\nỞ đây có sẵn tools / actions:\n\n<tools>\n{"name": "sum_two_elements", "description": "\\n    Computes the sum of two integers.\\n    Args:\\n        a (int): S\\u1ed1 nguy\\u00ean th\\u1ee9 nh\\u1ea5t \\u0111\\u1ec3 t\\u00ednh t\\u1ed5ng.\\n        b (int): S\\u1ed1 nguy\\

## ReAct Loop Step 5

In [139]:
output = GROQ_CLIENT.chat.completions.create(
    messages = chat_history,
    model = MODEL
).choices[0].message.content

print(output)


Tôi đã nhận được kết quả từ hàm `multiply_two_elements`. Kết quả là 34560.

Tiếp theo, tôi sẽ gọi hàm `compute_log` để tính logarithm của kết quả.

<tool_call>{"name": "compute_log", "arguments": {"x": 34560}, "id": 2}</tool_call>


In [142]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

## ReAct Loop Step 6

In [143]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])

tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [144]:
tool_result

10.450452222917992

In [145]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

In [146]:
chat_history

[{'role': 'system',
  'content': '\nBạn là một Function calling AI model. Bạn hoạt động theo một vòng lặp gồm 3 bước: Thought, Action, Observation.\nBạn được cung cấp một Function signatures với <tools></tools> XML tags.\nBạn có thể gọi một hoặc nhiều functions để hỗ trợ cùng với user query. Không được đưa ra những giả định về những giá trị nào sẽ được đưa vào Functions. Đặc biệt cần chú ý tới các đặc tính \'types\'. Bạn nên sử dụng types như trong một lệnh Python.\n\nMỗi Function Call trả về một đối tượng JSON cùng với Function Name và các Arguments within <tool_call></tool_call> XML tags as follows:\n<tool_call>\n{"name":<function-name>, "arguments":"args-dict", "id":<monotonically-increasing-id>}\n</tool_call>\n\nỞ đây có sẵn tools / actions:\n\n<tools>\n{"name": "sum_two_elements", "description": "\\n    Computes the sum of two integers.\\n    Args:\\n        a (int): S\\u1ed1 nguy\\u00ean th\\u1ee9 nh\\u1ea5t \\u0111\\u1ec3 t\\u00ednh t\\u1ed5ng.\\n        b (int): S\\u1ed1 nguy\\

## ReAct Loop Step 7

In [147]:
output = GROQ_CLIENT.chat.completions.create(
    messages = chat_history,
    model = MODEL
).choices[0].message.content

print(output)


Tôi đã nhận được kết quả từ hàm `compute_log`. Kết quả là 10.450452222917992.

Vậy là tôi đã hoàn thành yêu cầu của bạn. Tổng của hai số 1234 và 5678 là 6912. Nhân kết quả đó với 5 được 34560. Logarithm của kết quả đó là 10.450452222917992.

<response> Tổng của hai số 1234 và 5678 là 6912. Nhân kết quả đó với 5 được 34560. Logarithm của kết quả đó là 10.450452222917992. </response>


# Doing the same but with agentic_patterns

In [149]:
%cd 

/home/ngocson


In [148]:
from src.agentic_patterns.planning_pattern.react_agent import ReactAgent

ModuleNotFoundError: No module named 'agentic_patterns'

In [ ]:
agent = ReactAgent(tools=[sum_two_elements, multiply_two_elements, compute_log])

In [ ]:
agent.run(user_msg="Tôi muốn tính toán tổng của hai số 1234 với 5678 và nhân kết quả đó với 5. Sau đó tôi muốn tính logarithms cho kết quả đó")